In [1]:
import multiprocessing

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymorphy2

import re


In [2]:
def get_faq(urls,question_data,answer_data):
    #
    #  urls - list [urls1,urls2,..]
    #  question_data, answer_data - list [tag_value,class_value]  
    #
    #  return faq - list shape (n,2), where [:,0] - question, [:,1] - answer
    #
    question_tag,question_class=question_data
    answer_tag,answer_class=answer_data
    
    rs =[ requests.get(url) for url in urls]
    responses = [r.text.encode('utf-8') for r in rs]
    soups = [BeautifulSoup(response) for response in responses]

    faq=[]
    for soup in soups:
        question=soup.findAll(question_tag, class_=question_class)
        answer=soup.findAll(answer_tag,class_=answer_class)
        temp_questions=[]
        temp_answers=[]
        for q in question:
            txt=str(q.text)
            txt=txt.replace('  ','')
            txt=txt.replace('"','')
            txt=txt.replace('	','')
            txt=txt.replace('\n','')
            temp_questions.append(txt)
        for ans in answer:
            txt=str(ans.text)
            txt=txt.replace('  ','')
            txt=txt.replace('	','')
            txt=txt.replace('\n','')
            temp_answers.append(txt)
        
        temp_questions=np.array(temp_questions).reshape((-1,1))
        temp_answers=np.array(temp_answers).reshape((-1,1))
        temp_faq=np.concatenate([temp_questions,temp_answers],axis=1)
        faq.append(temp_faq)
            

    return faq

In [3]:
urls1=['https://www.mosenergosbyt.ru/common/lobby/questions/category_5749.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_5739.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_48100.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_48101.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_48102.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_48103.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_5745.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_5763.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_5754.php',
     'https://www.mosenergosbyt.ru/common/lobby/questions/category_5716.php']

urls2=['https://mosoblgaz.ru/company/query_answer/']

faq1=get_faq(urls1,['button','btn btn-link collapsed'],['div','collapse'])
faq2=get_faq(urls2,['div','faq-list-item__title js-accordion-item-title'],
             ['div','faq-list-item__text-inner js-accordion-item-inner'])


In [ ]:
df=[]
for document in faq1:
    for line in document:
        df.append(line)
    

df=pd.DataFrame(df,columns = ['Question','Answer'])
df.to_csv('faq.csv',index=False,encoding='utf-8')